In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix
from skimage.feature import graycomatrix, graycoprops
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Montăm Google Drive
drive.mount('/content/drive')

# Căile către directoare
train_dir = '/content/drive/MyDrive/train'
val_dir = '/content/drive/MyDrive/val'
test_dir = '/content/drive/MyDrive/test'

# Funcție pentru extragerea caracteristicilor bazate pe textura GLCM pe canale de culoare
def extract_texture_features(image, distances=[1], angles=[0], properties=['contrast', 'homogeneity']):
    features = []
    for channel in range(image.shape[2]):  # Numărul de canale (RGB)
        channel_image = image[:, :, channel]
        glcm = graycomatrix(channel_image, distances=distances, angles=angles, symmetric=True, normed=True)
        for prop in properties:
            features.append(graycoprops(glcm, prop).mean())
    return features

# Funcție pentru încărcarea imaginilor și extragerea caracteristicilor
def load_and_extract_features(image_dir, label, target_size=(350, 350)):
    features = []
    labels = []
    images = []
    image_names = []
    for img_name in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_name)
        try:
            img = load_img(img_path, target_size=target_size)  # Imagini color (RGB)
            img_array = img_to_array(img).astype('uint8')  # Convertim la uint8 pentru GLCM
            texture_features = extract_texture_features(img_array)  # Extragem caracteristicile pe canale
            features.append(texture_features)

            labels.append(label)
            images.append(img)
            image_names.append(img_name)
        except Exception as e:
            print(f"Eroare la imaginea {img_name}: {e}")
    return features, labels, images, image_names

# Funcție pentru încărcarea întregului set de date
def load_dataset(train_dir, val_dir, test_dir):
    # Date de antrenare
    metal_train_features, metal_train_labels, _, _ = load_and_extract_features(os.path.join(train_dir, 'metal'), 0)
    paper_train_features, paper_train_labels, _, _ = load_and_extract_features(os.path.join(train_dir, 'paper'), 1)

    # Date de validare
    metal_val_features, metal_val_labels, _, _ = load_and_extract_features(os.path.join(val_dir, 'metal'), 0)
    paper_val_features, paper_val_labels, _, _ = load_and_extract_features(os.path.join(val_dir, 'paper'), 1)

    # Date de testare
    metal_test_features, metal_test_labels, metal_images, metal_names = load_and_extract_features(os.path.join(test_dir, 'metal'), 0)
    paper_test_features, paper_test_labels, paper_images, paper_names = load_and_extract_features(os.path.join(test_dir, 'paper'), 1)

    # Concatenăm datele de antrenament
    X_train = np.vstack((metal_train_features, paper_train_features))
    y_train = np.hstack((metal_train_labels, paper_train_labels))

    # Concatenăm datele de validare
    X_val = np.vstack((metal_val_features, paper_val_features))
    y_val = np.hstack((metal_val_labels, paper_val_labels))

    # Concatenăm datele de testare
    X_test = np.vstack((metal_test_features, paper_test_features))
    y_test = np.hstack((metal_test_labels, paper_test_labels))
    images = metal_images + paper_images
    image_names = metal_names + paper_names

    return X_train, y_train, X_val, y_val, X_test, y_test, images, image_names

# Încărcăm datele
print("Se încarcă datele...")
X_train, y_train, X_val, y_val, X_test, y_test, images, image_names = load_dataset(train_dir, val_dir, test_dir)
print("Datele au fost încărcate cu succes!")

# Antrenăm modelul Naive Bayes
print("Se antrenează modelul Naive Bayes...")
model = GaussianNB()
model.fit(X_train, y_train)

# Facem predicțiile
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

# Evaluăm performanța
print("\nRaport de clasificare validare:")
print(classification_report(y_val, y_val_pred, target_names=['Metal', 'Paper']))

print("\nMatrice de confuzie validare:")
print(confusion_matrix(y_val, y_val_pred))

print("\nRaport de clasificare testare:")
print(classification_report(y_test, y_test_pred, target_names=['Metal', 'Paper']))

print("\nMatrice de confuzie testare:")
print(confusion_matrix(y_test, y_test_pred))

# Afișăm performanța
correct_predictions = [0, 0]
incorrect_predictions = [0, 0]

for true, pred in zip(y_val, y_val_pred):
    if true == pred:
        correct_predictions[true] += 1
    else:
        incorrect_predictions[true] += 1

categories = ["Metal", "Paper"]
x = np.arange(len(categories))
width = 0.35

fig, ax = plt.subplots(figsize=(8, 6))
bar1 = ax.bar(x - width/2, correct_predictions, width, label="Corecte", color='g')
bar2 = ax.bar(x + width/2, incorrect_predictions, width, label="Incorecte", color='r')

ax.set_xlabel("Categorie")
ax.set_ylabel("Număr de exemple")
ax.set_title("Performanța modelului pe setul de validare")
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()

for bar in bar1:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, height, f'{height}', ha='center', va='bottom')
for bar in bar2:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, height, f'{height}', ha='center', va='bottom')

plt.show()

# Afișăm imaginile și predicțiile
label_map = {0: "Metal", 1: "Paper"}

print("\nAfișăm imaginile din setul de testare cu etichetele lor...")
for i in range(len(images)):
    plt.imshow(images[i])
    true_label = label_map[y_test[i]]
    predicted_label = label_map[y_test_pred[i]]
    plt.title(f"Corect: {true_label}\nPrezis: {predicted_label}")
    plt.axis('off')
    plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')